# Musical intervals, consonances and dissonances, temperaments

Code to explore dyadic intervals and their degree of consonance, as well as different temperaments.

In [1]:
from math import log, exp
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter, FormatStrFormatter
from ipywidgets import interact, interactive_output
from IPython.display import display as wdisplay
import ipywidgets as widgets
import harmony
# $matplotlib

In [3]:
# initialization
plt.rc('figure', figsize=(20, 10))
k_harp = 0.45

In [4]:
# Prepare UI
def leftlayout(degree):
    return widgets.Layout(left='-%d%%' % degree)

def genUI_P():
    w_phys_label = widgets.HTML(value='<b><center>Physical parameters</center></b>')
    w_duration = widgets.FloatSlider(min=0.1, max=6, value=2, step=0.1, continuous_update=False, description='Duration [s]')
    w_harmonics = widgets.IntSlider(min=1, max=20, value=6, continuous_update=False, description='no. Harmonics',
                                    style={'description_width': 'initial'})
    w_h_model = widgets.RadioButtons(options=harmony.h_models, description='Harm. model')
    w_k_harp = widgets.FloatSlider(min=0.01, max=0.5, value=k_harp, step=0.01, continuous_update=False, description='k_harp')
    w_k_harp.layout.visibility = 'hidden'
    w_spectrogram = widgets.Checkbox(description='Show Spectrogram')

    def handle_h_model(change):
        w_k_harp.layout.visibility = 'visible' if change['new'] == 'harp' else 'hidden'

    def handle_addcb(change):
        w_miscb.layout.visibility = 'visible' if change['new'] else 'hidden'

    w_h_model.observe(handle_h_model, names='value')
    w_file_label = widgets.HTML(value='Filename prefix <i>(leave empty to not save)</i>')
    w_filename = widgets.Text(description='')

    return widgets.VBox([w_phys_label, w_duration, w_harmonics, w_h_model, w_k_harp,
                         w_spectrogram, w_file_label, w_filename],
                        layout=leftlayout(20)), {
        'duration': w_duration,
        'harmonics': w_harmonics,
        'h_model': w_h_model,
        'k_harp': w_k_harp,
        'spectrogram': w_spectrogram,
        'filename': w_filename,
    }

def genUI_M():
    w_mus_label = widgets.HTML(value='<b>&nbsp; &nbsp; &nbsp; Musical parameters</b>')
    w_temperament = widgets.RadioButtons(options=harmony.temperaments, description='Temperament')
    w_tonic = widgets.IntSlider(min=-11, max=62, value=0, continuous_update=False, description='Tonic')
    w_interval = widgets.IntSlider(min=0, max=62, value=7, continuous_update=False, description='Interval')
    w_c_int2 = widgets.Checkbox(description='Interval 2', layout=leftlayout(20))
    w_interval2 = widgets.IntSlider(min=1, max=62, value=4, continuous_update=False, layout=leftlayout(30))
    return widgets.VBox([w_mus_label, w_temperament, w_tonic, w_interval,
                         widgets.HBox([w_c_int2, w_interval2], layout=widgets.Layout(width='70%', align_content='stretch'))
                       ]), {
        'temperament': w_temperament,
        'tonic': w_tonic,
        'int1': w_interval,
        'c_int2': w_c_int2,
        'int2': w_interval2
       }

w_out = widgets.Output(layout=widgets.Layout(width='75%', left='25%'))
@w_out.capture()
def captureoutput(obj, method, *args):
    w_out.clear_output(wait=True)
    return getattr(obj, method)(*args)

## Dyads and triads

In [5]:
def interactive_dyads_triads(temperament, tonic, int1, c_int2, int2, duration, harmonics,
                             h_model, k_harp, spectrogram, filename):
    c = harmony.IChord(temperament, tonic, duration, harmonics, False, h_model, k_harp, 0)
    # generate signal for the given interval
    if int1 == 0:
        s = c.note(tonic)
        d = 0
    elif not c_int2:
        s, d = captureoutput(c, 'dichord', int1)
    else:
        s, d = captureoutput(c, 'triad', int1, int2)
    c.play(s)
    c.plottimefreq(s, spectrogram)
    if filename:
        # harmonics = 10, h_model = harp-verotta for the test sounds
        c.save(s, '%s_%d_%s.wav' % (filename, tonic, temperament))
    # wait for playback to finish before exiting
    #p.wait_done()

uiM = genUI_M()
uiP = genUI_P()
ui = widgets.VBox([widgets.HBox([uiM[0], uiP[0]], layout=widgets.Layout(justify_content='center')), w_out])
interactive = interactive_output(interactive_dyads_triads, uiM[1] | uiP[1])
wdisplay(ui, interactive)

Output()